In [1]:
"""將tensorflow檔轉成tensorflow lite，使機器負載量變小"""
from tensorflow import lite
from tensorflow.keras import models

# Parameters
keras_model_filename = './h5/appendEnergyTT.h5' #訓練好的模型
tflite_filename = './tflite/appendEnergyTT.tflite' #預建置檔案

# Convert model to TF Lite model
model = models.load_model(keras_model_filename) #載入本來的模型
converter = lite.TFLiteConverter.from_keras_model(model) #將模型載入轉換器
tflite_model = converter.convert() #進行轉換
open(tflite_filename, 'wb').write(tflite_model) #輸出轉換後的模型


INFO:tensorflow:Assets written to: C:\Users\r6017\AppData\Local\Temp\tmp7e7nifmb\assets


286244

In [ ]:
#num_mfcc = 23，rec_duration = 0.5
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa
#import RPi.GPIO as GPIO

# Parameters
debug_time = 0 #Debug用
debug_acc = 1 #Debug用
led_pin = 8 #LED PIN
word_threshold = 0.5 #預測值>0.5，表示stop
rec_duration = 0.5 #每一段錄音持續時間
#window_stride = 0.5
sample_rate = 48000 #取樣率(依MIC不同而改變)
resample_rate = 8000 #重整後的取樣率(符合MODEL)
num_channels = 1 #音訊深度
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/appendEnergyTT.tflite'
words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞

s = 0 #秒
m = 0 #分
h = 0 #時

# Sliding window
window = np.zeros(int(rec_duration * resample_rate) * 2)#取樣音頻數據變數

# GPIO 
#GPIO.setwarnings(False)
#GPIO.setmode(GPIO.BOARD)
#GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#開始計時
counter = QTime()
counter.restart()

# Decimate (filter and downsample)
def decimate(signal, old_fs, new_fs):
    
    #檢查是否降低音頻
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs
    
    #檢查是否為整數(只能在整數下執行)
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs

    # Do decimation
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))
    
    return resampled_signal, new_fs

# This gets called every 0.5 seconds
def sd_callback(rec, frames, time, status):

    #GPIO.output(led_pin, GPIO.LOW)
    
    # Notify if errors
    if status:
        print('Error:', status)
    
    # Remove 2nd dimension from recording sample
    #壓縮成1D張量
    rec = np.squeeze(rec)
    
    # Resample
    #重取樣成8000HZ(以符合訓練模型)
    rec, new_fs = decimate(rec, sample_rate, resample_rate)
    
    # Save recording onto sliding window
    #將音訊輸入到window
    window[:len(window)//2] = window[len(window)//2:]
    window[len(window)//2:] = rec
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    #print("np.sum(S):" + str(np.sum(S)))
    
    if np.sum(S) > 3000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=new_fs, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=True,#係數0的話對被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if(list_val_max >= 0.3):#如果預測值>=0.3
            print(words[list_val_maxIndex])#輸出相對應的字詞
            print("MAX:" + str(list_val_max))#輸出預測值當中最大的值
            print(str(h) + "時" + str(m) + "分" + "{:.1f}秒".format(s))              
    
    
# Start streaming from microphone
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        s = float(counter.elapsed() / 1000)
        if(s >= 60):
            counter.restart()
            s = 0
            m = m + 1
        if(m == 60):
            m = 0
            h = h + 1
        pass


啦
MAX:0.9248993992805481
0時0分2.5秒
啦
MAX:0.5260702967643738
0時0分2.9秒
ㄏㄧㄡ
MAX:0.7713937759399414
0時0分4.8秒
啦
MAX:0.46423250436782837
0時0分5.4秒
ㄏㄧㄡ
MAX:0.6896242499351501
0時0分7.5秒
那那個
MAX:0.5548799633979797
0時0分7.7秒
阿
MAX:0.6336732506752014
0時0分10.0秒
吼
MAX:0.7982702851295471
0時0分10.9秒
吼
MAX:0.5394558310508728
0時0分12.5秒
啦
MAX:0.9552400708198547
0時0分13.6秒
吼
MAX:0.6354144215583801
0時0分15.4秒
啦
MAX:0.7938937544822693
0時0分15.8秒
ㄏㄧㄡ
MAX:0.6473138928413391
0時0分16.5秒
著
MAX:0.9310646057128906
0時0分18.0秒
嗯
MAX:0.48291143774986267
0時0分18.4秒
的這個
MAX:0.4381025433540344
0時0分21.0秒
的那個
MAX:0.8886422514915466
0時0分21.6秒
的那個
MAX:0.5200534462928772
0時0分22.5秒
的那個
MAX:0.967972457408905
0時0分22.9秒
啦
MAX:0.815345287322998
0時0分24.5秒
阿
MAX:0.694421648979187
0時0分25.0秒
ㄟ
MAX:0.30966120958328247
0時0分25.5秒
嗯
MAX:0.4106645882129669
0時0分26.0秒
ㄟ
MAX:0.4693686366081238
0時0分26.7秒
啦
MAX:0.41132664680480957
0時0分27.6秒
啦
MAX:0.7788118124008179
0時0分28.1秒
的一個
MAX:0.9041814804077148
0時0分29.6秒
的一個
MAX:0.906627893447876
0時0分30.1秒
啦
MAX:

In [3]:
#num_mfcc = 23，rec_duration = 0.25
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa
#import RPi.GPIO as GPIO

# Parameters
debug_time = 0 #Debug用
debug_acc = 1 #Debug用
led_pin = 8 #LED PIN
word_threshold = 0.5 #預測值>0.5，表示stop
rec_duration = 0.25 #每一段錄音持續時間
#window_stride = 0.5
sample_rate = 48000 #取樣率(依MIC不同而改變)
resample_rate = 8000 #重整後的取樣率(符合MODEL)
num_channels = 1 #音訊深度
num_mfcc = 23 #回傳mfcc的量
model_path = './tflite/recording23.tflite'
words = ['ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞

s = 0 #秒
m = 0 #分
h = 0 #時

# Sliding window
window = np.zeros(int(rec_duration * resample_rate) * 4)#取樣音頻數據變數

# GPIO 
#GPIO.setwarnings(False)
#GPIO.setmode(GPIO.BOARD)
#GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

#開始計時
counter = QTime()
counter.restart()

# Decimate (filter and downsample)
def decimate(signal, old_fs, new_fs):
    
    #檢查是否降低音頻
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs
    
    #檢查是否為整數(只能在整數下執行)
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs

    # Do decimation
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))
    
    return resampled_signal, new_fs

# This gets called every 0.5 seconds
def sd_callback(rec, frames, time, status):

    #GPIO.output(led_pin, GPIO.LOW)
    
    # Notify if errors
    if status:
        print('Error:', status)
    
    # Remove 2nd dimension from recording sample
    #壓縮成1D張量
    rec = np.squeeze(rec)
    
    # Resample
    #重取樣成8000HZ(以符合訓練模型)
    rec, new_fs = decimate(rec, sample_rate, resample_rate)
    
    # Save recording onto sliding window
    #將音訊輸入到window
    window[:2000] = window[2000:4000]
    window[2000:4000] = window[4000:6000]
    window[4000:6000] = window[6000:]
    window[6000:] = rec
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    #print("np.sum(S):" + str(np.sum(S)))

    if np.sum(S) > 3000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=new_fs, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=False,#係數0的話對被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        if(list_val_max >= 0.3):#如果預測值>=0.3
            print(words[list_val_maxIndex])#輸出相對應的字詞
            print("MAX:" + str(list_val_max))#輸出預測值當中最大的值
            print(str(h) + "時" + str(m) + "分" + "{:.1f}秒".format(s))
    
# Start streaming from microphone
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        s = float(counter.elapsed() / 1000)
        if(s >= 60):
            counter.restart()
            s = 0
            m = m + 1
        if(m == 60):
            m = 0
            h = h + 1
        pass


Error: input overflow
Error: input overflow
Error: input overflow
Error: input overflow


KeyboardInterrupt: 